In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
from bs4 import BeautifulSoup
import html5lib
import requests
import datetime
import time
from f1 import Team
from f1 import Race
import matplotlib.animation as animation

In [23]:
#Change these variables to update the race:
RACE = '8'
RACENAME = 'Azerbaijan'
# RACE = '7'
# RACENAME = 'Canada'

In [ ]:
#This section is to download fresh results from Ergast
path = 'http://ergast.com/api/f1'
season = '2016'
URL = path + '/' + season + '/' + RACE + '/' + 'laps?limit=1000'

#Download
results = requests.get(URL)
#soup = BeautifulSoup(results.content, 'lxml')
print("First 1000 records downloaded")
#Part 2
URL = path + '/' + season + '/' + RACE + '/' + 'laps?limit=1000&offset=1000'
results2 = requests.get(URL)
#soup2 = BeautifulSoup(results2.content, 'lxml')
with open(RACENAME + '_race_dl.xml', 'wb') as f:
    f.write(results.content)
    f.write(results2.content)

In [24]:
#This section will just open a saved copy.  Run it always to pull the composite soup
with open(RACENAME + '_race_dl.xml', 'rb') as f:
    results = f.read()
soup = BeautifulSoup(results, 'lxml')

In [25]:
#pull in driverids to names
driver_info = pd.DataFrame.from_csv('driver_info.csv')
driverid_dict = {driver_info.ix[i]['driverid']:driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName'] for i in driver_info.index}
driverid_dict['sainz'] = driverid_dict['sainz'] + ' Jr.'
colors = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']: driver_info.ix[i]['color'] for i in driver_info.index}
#Manually Correct Carlos Sainz
colors['Carlos Sainz Jr.'] = colors['Carlos Sainz']
driver_abrev = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']:driver_info.ix[i]['code'] for i in driver_info.index}
driver_abrev['Carlos Sainz Jr.'] = 'SAI'

In [26]:
headers = ('driverid', 'lap', 'time')
num_rows = len(soup.find_all('timing'))
race_dict = {}
for header in headers:
    race_dict[header] = [race[header] for race in soup.find_all('timing')]
df = pd.DataFrame(race_dict)
df['minutes'] = df['time'].apply(lambda x: x.split(':')[0])
df['seconds'] = df['time'].apply(lambda x: x.split(':')[1])
df['lap_sec'] = df['minutes'].astype('int64') * 60 + df['seconds'].astype('float32')
df['lap'] = df['lap'].astype('int64')
df['driver'] = df['driverid'].apply(lambda x: driverid_dict[x])

In [27]:
pivot = df.pivot(index='driver', columns='lap', values='lap_sec')

In [28]:
for i in range(2, df['lap'].max() + 1):
    pivot.ix[:,i] = pivot.ix[:,i] + pivot.ix[:, i-1]

In [29]:
unpivot = pivot.stack()
new_df = unpivot.reset_index()
new_df.columns = ['driverid', 'lap', 'cume_sec']
new_df.sort_values('cume_sec', ascending=True, inplace=True)
new_df = new_df.reset_index()
new_df.drop('index', axis=1, inplace=True)

In [30]:
#Set up a Race object, score it, return a dataframe of all points
ThisRace = Race(RACENAME)
df_score = ThisRace.score()
ThisTeam = Team(RACENAME)

In [34]:
#Set up current dataframe:
#tracks each drivers position at a certain point of the race
#and the laps completed.
current = pd.DataFrame(columns=['position', 'lap'])
current['position'] = df_score['Grid'].copy()
current['lap'] = 0
if 'DNS' in current['position']:
    current['position'][current['position'] == 'DNS'] = np.NaN
if 'PL' in current['position']:
    current['position'][current['position'] == 'PL'] = np.NaN

#Fix pit lane
current['position']['Kevin Magnussen'] = 22

current.sort_values('position', inplace=True)

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
#Set all laps completed to zero
ThisRace.laps_completed = {driver:current.ix[driver]['lap'] for driver in current.index}

In [38]:
#Reworking on real time
fastest_lap_time = 1000 #dummy value
first = False 
row_count = 0
fps = 20

def blank():
    pass
def update_driver_scores2(step, rects, ax):
    global fastest_lap_time
    global ThisRace
    global current
    global new_df
    global first
    global row_count
    global fps
    #print('Step:', step, 'Row Count:', row_count)
    if float(new_df.ix[row_count]['cume_sec']) < step / fps:
        driver, lap, cume_sec = new_df.ix[row_count]
        if lap == 1:
            if cume_sec != np.NaN:
                cur_lap_time = cume_sec
        elif new_df.ix[row_count, 'cume_sec'] != np.NaN:
            cur_lap_time = cume_sec - float(new_df['cume_sec'][(new_df['driverid'] == driver) & (new_df['lap'] == lap - 1)])
        if cur_lap_time < fastest_lap_time: #check for fastest lap
            ThisRace.fastest_lap = driver
            fastest_lap_time = cur_lap_time
        if cume_sec == np.NaN:
            #ThisRace.drivers_position[driver] = "Ret"
            #print(driver, 'is out!')
            xxx = 0 #filler code
        else:
            current.ix[driver, 'lap'] += 1 #increment lap
            current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable sort
            #print('before: \n', current)
            current['position'] = np.arange(1,len(current) + 1) #recalc positions
            #print('after: \n', current)
            ThisRace.laps_completed[driver] = lap #update laps completed for driver
            ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
        ThisRace.score().sort_values('Finish', inplace=True, ascending=True)
        #print(ThisRace.fantasy_points)
        for rect, h, driver in zip(rects, 
                                   [ThisRace.fantasy_points[driver] for driver in current.index], 
                                   [driver for driver in current.index]):
            #print(rect, h, driver)
            rect.set_height(h)
            rect.set_color(colors[driver])
            rect.set_edgecolor('#000000')
        ax.xaxis.set_ticklabels([driver_abrev[driver] for driver in current.index])
        #print([driver for driver in current.index])
        #ax.xaxis.set_ticklabels(current.index)
        row_count += 1
        

#whole race
start_time = datetime.datetime.now()
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()


fig1 = plt.figure()
#fig, ax = plt.subplots()
ax = fig1.add_subplot(1,1,1)
#ax.grid(False)
ax.xaxis.grid(False)
ax.yaxis.grid(True)
ax.set_frame_on(False)
ax.tick_params(axis='both', width=0)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = ax.bar(x_pos, points, align='center', edgecolor='#000000', linewidth=0.5)
plt.xticks(x_pos, [driver_abrev[driver] for driver in current.index], rotation=90)
plt.xlim([0, len(points)+1])
plt.title('Drivers Fantasy Points')
plt.ylabel('Points')
plt.ylim([0, 25])
interval = int(new_df[-1:]['cume_sec'] * fps)
# ax.spines['top'].set_visible(True)
# ax.spines['top'].set_linewidth(3)
for rect, h, driver in zip(rects, 
                           [ThisRace.fantasy_points[driver] for driver in current.index], 
                           [driver for driver in current.index]):
            rect.set_height(h)
            rect.set_color(colors[driver])
            rect.set_edgecolor('#000000')
ani = animation.FuncAnimation(fig1, update_driver_scores2, interval, fargs=([rects, ax]), init_func=blank)
ani.save(RACENAME + '9_movie.mp4', fps=fps, writer='ffmpeg', bitrate=5000)